# Hosting a Live Endpoint for Real-Time Inference with SageMaker for Customer Churn

## Environment Setup

- Image: Data Science
- Kernel: Python 3
- Instance type: ml.t3.medium

## Background

This notebook builds on previous notebooks where we trained a model to predicts customer churn (i.e., when a company loses a customer).  In this iteration of the notebook, we deploy our trained model to a live endpoint then pass in test data to see how well the model performs predictions.

To keep things simple, Experiments have been removed from this version of the notebook.

This notebook has been adapted from the [SageMaker examples](https://github.com/aws/amazon-sagemaker-examples/blob/main/introduction_to_applying_machine_learning/xgboost_customer_churn/xgboost_customer_churn.ipynb).

## Initialize Environment and Variables

In [ ]:
# Import libraries
import boto3
import re
import pandas as pd
import numpy as np
import os
import time
import io

import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import CSVSerializer
from sagemaker.inputs import TrainingInput

# Get the SageMaker session and the execution role from the SageMaker domain
sess = sagemaker.Session()
role = get_execution_role()

bucket = '<name-of-your-bucket>' # Update with the name of a bucket that is already created in S3
prefix = 'demo' # The name of the folder that will be created in the S3 bucket

## Data

For this lesson, data has already been cleaned and split into two local CSV files: **train.csv** (used to train the model) and **validation.csv** (used to validate how well the model does).

We'll take these local files and upload them to S3 so SageMaker can use them.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

## Train

We trained the model in previous lessons, but to make it easier to follow along with this notebook, we'll do that again here.

In this section, we need to specify three things: where our training data is, the path to the algorithm container stored in the Elastic Container Registry, and the algorithm to use (along with hyperparameters).

The training job (the Estimator) takes in several hyperparameters.  More information on the hyperparameters for the XGBoost algorithm can be found [here](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html).

In [ ]:
# The location of our training and validation data in S3
s3_input_train = TrainingInput(
    s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv'
)
s3_input_validation = TrainingInput(
    s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv'
)

In [ ]:
# The location of the XGBoost container version 1.5-1 (an AWS-managed container)
container = sagemaker.image_uris.retrieve('xgboost', sess.boto_region_name, '1.5-1')

In [ ]:
# Initialize hyperparameters
hyperparameters = {
                    'max_depth':'5',
                    'eta':'0.2',
                    'gamma':'4',
                    'min_child_weight':'6',
                    'subsample':'0.8',
                    'objective':'binary:logistic',
                    'eval_metric':'error',
                    'num_round':'100'}

# Output path where the trained model will be saved
output_path = 's3://{}/{}/output'.format(bucket, prefix)

# Set up the Estimator, which is training job
xgb = sagemaker.estimator.Estimator(image_uri=container, 
                                    hyperparameters=hyperparameters,
                                    role=role,
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge', 
                                    output_path=output_path,
                                    sagemaker_session=sess)

In [ ]:
# "fit" executes the training job
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

## Host

Now that we've trained the model, let's deploy it to an endpoint so we can send data to it for live prediction.  We can do that with a single line.  This call to "deploy" will create our endpoint configuration and endpoint all at the same time.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

## Make Predictions

Now that our endpoint is live, let's pass in test data to get predictions.

In [ ]:
# Read test data into a dataframe and transform it into a CSV that can be passed to the endpoint
payload = pd.read_csv('test.csv')
payload_file = io.StringIO()
payload.to_csv(payload_file, header = None, index = None)

In [ ]:
# Create a low-level client for the SageMaker Runtime
sagemaker_runtime = boto3.client('sagemaker-runtime')

# Client applications use this API to get inferences from the hosted model, by calling invoke_endpoint
# We pass in the test data/payload file we read in earlier
response = sagemaker_runtime.invoke_endpoint(
                            EndpointName=xgb_predictor.endpoint_name, 
                            Body=payload_file.getvalue(),
                            ContentType = 'text/csv'
)

# Print the response body, which contains the predictions
print(response['Body'].read().decode('utf-8'))